In [1]:
import os 
from huggingface_hub import hf_hub_download
import transformers
import torch

In [2]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("timdettmers/openassistant-guanaco", split = "train")

ds = dataset.select(range(500))

Repo card metadata block was not found. Setting CardData to empty.


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [10]:
data=[]
for x in ds:
    a = x['text']
    data.append(a)

In [11]:
data[0]

'### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power, lead

In [12]:
from transformers import BertTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text data
tokens = [tokenizer.tokenize(text) for text in data]

# Filter out instructions with less than 100 tokens
filtered_tokens = [token for token in tokens if len(token) >= 100]

# Convert tokens to text
text_corpus = [" ".join(token) for token in filtered_tokens]

# Create TF-IDF vectors
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(text_corpus)

# Calculate cosine similarity matrix
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Define the similarity threshold
similarity_threshold = 0.95

# Group similar instructions
groups = []
visited = [False] * len(text_corpus)
for i in range(len(text_corpus)):
    if not visited[i]:
        group = [i]
        visited[i] = True
        for j in range(i + 1, len(text_corpus)):
            if not visited[j] and cosine_sim_matrix[i, j] >= similarity_threshold:
                group.append(j)
                visited[j] = True
        groups.append(group)



In [13]:
# Print the grouped instructions and their input IDs
text = []
input_ids = []
print("Grouped Instructions:")
for group in groups:
    for idx in group:
        text.append(text_corpus[idx])
        input_ids.append(tokenizer.convert_tokens_to_ids(filtered_tokens[idx]))

Grouped Instructions:


In [14]:
len(text)

464

In [21]:
csvdata=[]
for i in range(464):
    csvdata.append([text[i]])

In [22]:
import csv
with open('data.csv', 'w', newline='') as csvfile:
  writer = csv.writer(csvfile)

  # Write header row (optional)
  writer.writerow(['text'])

  # Write data rows
  writer.writerows(csvdata)

print("CSV file created successfully!")

CSV file created successfully!
